## CS345 Project

For our dataset, we will be using the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). Due to the size of this dataset, it could not be uploaded to github. Please download the dataset yourself, extract it, and move it to the "data" directory.

In [1]:
print("Hello World")

Hello World
